# CJE (Causal Judge Evaluation) - Quick Start

**Get started with CJE in 5 minutes using Arena 5K sample data (1000 prompts)**

This notebook shows the simplest way to evaluate LLM policies:

1. **Setup**: Install CJE and download sample data
2. **Inspect Data**: Understand the dataset structure  
3. **Direct Mode**: Compare policies on an evaluation set (no logprobs needed!)
4. **Policy Selection**: Find the best policy with statistical tests
5. **Visualization**: Plot results

---

## What is CJE?

CJE turns LLM-as-judge scores into calibrated policy estimates:
- **AutoCal-R**: Calibrates cheap judge scores to expensive oracle labels
- **Valid inference**: Confidence intervals account for all uncertainty

**Key insight**: Raw judge scores are correlational. CJE makes them causal.

**This tutorial covers Direct Mode** - the simplest way to compare policies. For advanced off-policy evaluation (IPS and DR modes), see [`cje_advanced.ipynb`](cje_advanced.ipynb).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cimo-labs/cje/blob/main/examples/cje_tutorial.ipynb)

## Step 1: Setup

Install CJE and download the Arena sample data.

**Note:** Colab comes with numpy 2.3+ pre-installed, which breaks scipy. We force-reinstall numpy 2.0.x first.

In [ ]:
# Colab comes with numpy 2.3+ which breaks scipy
# Force install compatible numpy first
!pip install -q 'numpy>=2.0,<2.1' --force-reinstall

# Install latest CJE from PyPI with cache busting
!pip install --no-cache-dir --upgrade cje-eval

# Verify installation
import cje
print(f"✓ CJE version {cje.__version__} installed")

# Check numpy version
import numpy as np
print(f"✓ NumPy version {np.__version__}")

In [ ]:
# Download Arena sample data from GitHub
import os
import urllib.request
from pathlib import Path

# Create data directory
DATA_DIR = Path("arena_sample")
DATA_DIR.mkdir(exist_ok=True)
(DATA_DIR / "fresh_draws").mkdir(exist_ok=True)

# Base URL for raw files on GitHub
BASE_URL = "https://raw.githubusercontent.com/cimo-labs/cje/main/examples/arena_sample"

# Download fresh draws for Direct mode
fresh_draw_files = {
    "base": "base_responses.jsonl",
    "clone": "clone_responses.jsonl",
    "parallel_universe_prompt": "parallel_universe_prompt_responses.jsonl",
    "unhelpful": "unhelpful_responses.jsonl"
}

for policy, filename in fresh_draw_files.items():
    print(f"Downloading fresh_draws/{filename}...")
    urllib.request.urlretrieve(
        f"{BASE_URL}/fresh_draws/{filename}",
        DATA_DIR / "fresh_draws" / filename
    )
    print(f"✓ Downloaded {filename}")

print("\n✓ All data downloaded successfully!")
print(f"\nData location: {DATA_DIR.absolute()}")
print(f"Policies: {list(fresh_draw_files.keys())}")

## Step 2: Inspect the Data

Let's look at what we're working with.

In [ ]:
import json

# Load and inspect fresh draws (base policy)
with open(DATA_DIR / "fresh_draws" / "base_responses.jsonl") as f:
    base_fresh_draws = [json.loads(line) for line in f]

print(f"Base policy fresh draws: {len(base_fresh_draws)} samples")
print(f"\nExample sample:")
sample = base_fresh_draws[0]
print(f"  Prompt: {sample['prompt'][:100]}...")
print(f"  Response: {sample['response'][:150]}...")
print(f"  Judge score: {sample['judge_score']}")
oracle_label = sample.get('oracle_label', 'N/A')
print(f"  Oracle label: {oracle_label}")

# Check oracle coverage
n_base_oracle = sum(1 for s in base_fresh_draws if s.get('oracle_label') is not None)
base_coverage = n_base_oracle / len(base_fresh_draws)
print(f"\nOracle label coverage: {n_base_oracle}/{len(base_fresh_draws)} ({base_coverage:.1%})")

print(f"\n💡 CJE uses these oracle labels for AutoCal-R calibration:")
print(f"   1. Learn mapping: judge_score → oracle_label (from base policy)")
print(f"   2. Apply calibration to all policies for fair comparison")

## Step 3: Direct Mode - Policy Comparison

**Direct Mode** is the simplest way to compare policies:

**Use case**: "Which policy performs best on this evaluation set?"

**How it works**:
1. Generate fresh responses from each policy on the same prompts
2. Calibrate judge scores using oracle labels (AutoCal-R)
3. Average calibrated scores: V̂(π) = (1/m) Σ R_πi

**Key advantages**:
- ✓ No logprobs needed
- ✓ Simplest mode
- ✓ Works with any LLM (even closed-source APIs)
- ✓ Just 5-10% oracle labels often enough

**When to use**: Quick policy comparisons, A/B testing, leaderboards

In [ ]:
from cje import analyze_dataset

# Run Direct mode analysis
results = analyze_dataset(
    fresh_draws_dir=str(DATA_DIR / "fresh_draws"),
    estimator="auto",  # Auto-detects Direct mode
    verbose=True,
)

print("\n" + "="*70)
print("Direct Mode Results")
print("="*70)
print(f"Mode: {results.metadata['mode']}")
print(f"Estimator: {results.metadata['estimator']}")
print(f"Calibration: {results.metadata.get('calibration', 'none')}")
print(f"Oracle coverage: {results.metadata.get('oracle_coverage', 0):.1%}")
print()

# Show estimates with confidence intervals
policies = results.metadata['target_policies']
print(f"{'Policy':<30} {'Estimate':<12} {'Std Error':<12} {'95% CI':<20}")
print("-" * 74)
for i, policy in enumerate(policies):
    est = results.estimates[i]
    se = results.standard_errors[i]
    ci_low = est - 1.96 * se
    ci_high = est + 1.96 * se
    print(f"{policy:<30} {est:>6.3f}       {se:>6.3f}       [{ci_low:.3f}, {ci_high:.3f}]")

## Step 4: Policy Selection

Now let's find the best policy and compare it to a baseline.

In [ ]:
import numpy as np

# Get results
policies = results.metadata['target_policies']
estimates = np.array(results.estimates)
std_errors = np.array(results.standard_errors)

# Find best policy
best_idx = results.best_policy()
best_policy = policies[best_idx]
best_est = estimates[best_idx]
best_se = std_errors[best_idx]

print("Policy Selection")
print("="*70)
print(f"\n🏆 Best policy: {best_policy}")
print(f"   Estimate: {best_est:.3f} ± {best_se:.3f}")
print(f"   95% CI: [{best_est - 1.96*best_se:.3f}, {best_est + 1.96*best_se:.3f}]")

# Compare against baseline (clone)
baseline_policy = 'clone'
baseline_idx = policies.index(baseline_policy)

print(f"\n📊 Comparison to baseline ({baseline_policy}):")
print(f"   Baseline: {estimates[baseline_idx]:.3f} ± {std_errors[baseline_idx]:.3f}")
print()
print(f"{'Policy':<30} {'Delta':<12} {'SE(Δ)':<12} {'p-value':<12} {'Significant?':<15}")
print("-" * 81)

for i, policy in enumerate(policies):
    if i == baseline_idx:
        print(f"{policy:<30} {'(baseline)':<12} {'':<12} {'':<12} {'':<15}")
        continue
    
    # Use built-in comparison (uses influence functions for paired variance)
    comp = results.compare_policies(i, baseline_idx)
    
    sig_text = "✓ Yes (p<0.05)" if comp['significant'] else "No"
    
    print(f"{policy:<30} {comp['difference']:+.3f}       "
          f"{comp['se_difference']:.3f}       "
          f"{comp['p_value']:.3f}      {sig_text:<15}")

print("\n💡 CJE uses influence functions for paired comparisons,")
print("   accounting for correlation between estimates on the same prompts.")

## Step 5: Visualization

### HTML Display

In Jupyter notebooks, results automatically display as formatted tables:

In [ ]:
# Just evaluate the result object to see a nice HTML table
results

### Forest Plot

Create a forest plot with confidence intervals:

In [ ]:
import matplotlib.pyplot as plt

# Use the convenience method
fig = results.plot_estimates()
plt.tight_layout()
plt.show()

print("\n💡 Green dot = best policy")

### Advanced Visualization

You can also use standalone plot functions from `cje`:

In [ ]:
from cje import plot_policy_estimates

# Extract estimates as dictionaries
estimates_dict = {policy: float(results.estimates[i]) for i, policy in enumerate(policies)}
ses_dict = {policy: float(results.standard_errors[i]) for i, policy in enumerate(policies)}

# Create forest plot
fig = plot_policy_estimates(
    estimates=estimates_dict,
    standard_errors=ses_dict,
    figsize=(10, 5)
)

plt.tight_layout()
plt.show()

print("\n✓ Forest plot shows point estimates with 95% confidence intervals")

## Summary

You've learned how to:
- ✓ Run Direct mode policy evaluation (no logprobs needed!)
- ✓ Find the best policy with statistical tests
- ✓ Visualize results with confidence intervals

### When to Use Direct Mode

**Perfect for**:
- Quick policy comparisons
- A/B testing
- Leaderboard evaluation
- APIs without logprob access

**Limitations**:
- Evaluates performance on *this specific eval set*
- Doesn't answer counterfactual: "What if we deployed policy π'?"
- For counterfactual estimates, use IPS or DR modes

### Next Steps

**Want off-policy (counterfactual) evaluation?**
- 📚 See [`cje_advanced.ipynb`](cje_advanced.ipynb) for IPS and DR modes
- Learn about importance sampling and doubly robust estimation
- Understand when DR gives tighter confidence intervals than IPS

**Documentation**:
- [CJE README](https://github.com/cimo-labs/cje)
- [API Reference](https://github.com/cimo-labs/cje/tree/main/cje/interface)
- [Blog Post: Arena Experiment](https://www.cimolabs.com/blog/arena-experiment)

**Install locally**: `pip install cje-eval`

Questions? Open an issue on [GitHub](https://github.com/cimo-labs/cje/issues)!